<a href="https://colab.research.google.com/github/harshith7823/CS760-Project/blob/reetu/part-1/Death%20prediction/Model%20generated%20death%20column/part1_model_based.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from numpy import mean
from numpy import std
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, ElasticNetCV,RidgeCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV

In [2]:
oppScrData = pd.read_excel(r'sample_data/OppScrData.xlsx')

In [3]:
def clean_ct_data(oppScrData):
    # Delete rows with empty values
    cols = ["L1_HU_BMD", "TAT Area (cm2)", 'Total Body                Area EA (cm2)',
       'VAT Area (cm2)', 'SAT Area (cm2)', 'VAT/SAT     Ratio', 'Muscle HU',
       ' Muscle Area (cm2)', 'L3 SMI (cm2/m2)', 'AoCa        Agatston',
       'Liver HU    (Median)', 'DEATH [d from CT]']
    ct_data= oppScrData[cols]
    n = ct_data.shape[0]
    preprocessed_ct_data = pd.DataFrame( columns=cols)
    for i in range(n):
        row = ct_data.loc[i]
        ignore = False
        for k,j in enumerate(row):
          if pd.isna(j) or j == ' ': 
            if k == 11:              
              ignore = False
            else:
              ignore = True
              break
        if not ignore:
          preprocessed_ct_data.loc[i]= row
    return preprocessed_ct_data

In [4]:
sc = StandardScaler()
ct_data = clean_ct_data(oppScrData=oppScrData)

In [5]:
non_null_death_rows = ct_data[~ct_data['DEATH [d from CT]'].isnull()]
print(non_null_death_rows.shape)
print(non_null_death_rows.columns)

(493, 12)
Index(['L1_HU_BMD', 'TAT Area (cm2)',
       'Total Body                Area EA (cm2)', 'VAT Area (cm2)',
       'SAT Area (cm2)', 'VAT/SAT     Ratio', 'Muscle HU',
       ' Muscle Area (cm2)', 'L3 SMI (cm2/m2)', 'AoCa        Agatston',
       'Liver HU    (Median)', 'DEATH [d from CT]'],
      dtype='object')


In [6]:
# Convert to years
non_null_death_rows['DEATH [d from CT]'] /= 365

X = non_null_death_rows.iloc[:, 0:11]
y = non_null_death_rows.iloc[:, 11]
X = pd.DataFrame(sc.fit_transform(X), columns = X.columns)

null_death_rows=ct_data[ct_data['DEATH [d from CT]'].isnull()]
X2 = null_death_rows.iloc[:, 0:11]
X2 = pd.DataFrame(sc.fit_transform(X2), columns = X2.columns)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [7]:
X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size=0.2, random_state=0)

In [70]:
def testWithModel(model, X_train, X_test, y_train, y_test):
  clf = model
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)

  print(f'MAE = {mean_absolute_error(y_test, y_pred)}') 
  print(f'MAPE = {mean_absolute_percentage_error(y_test, y_pred)}')
  print(f'RMSE = {mean_squared_error(y_test, y_pred)**0.5}')

  # plot_results(y_test, y_pred)

  return y_pred

In [52]:
RFparams2={'n_estimators':[50],
          'min_samples_split':[16],
          'min_samples_leaf': [4],
           'max_features': [6],
           'max_depth': [20],
           'bootstrap': [True]}

rf = RandomForestRegressor(random_state = 21)
clf_random_forest = GridSearchCV(rf, RFparams2, refit=True)
testWithModel(clf_random_forest ,X_train, X_test, y_train, y_test)

MAE = 0.3177016904556609
MAPE = 0.10556548814159779
RMSE = 1.0095846946554512


In [53]:
testWithModel(Lasso(alpha=0.5), X_train, X_test, y_train, y_test)

MAE = 0.31166511904137817
MAPE = 0.10179816634800201
RMSE = 0.9968911490930105


In [54]:
testWithModel(RidgeCV(),X_train, X_test, y_train, y_test)

MAE = 0.31296540640937887
MAPE = 0.10209742913003998
RMSE = 0.9950907017053464


In [55]:
testWithModel(KNeighborsRegressor(),X_train, X_test, y_train, y_test)

MAE = 0.34097741475070614
MAPE = 0.11040402555178135
RMSE = 1.0872826525188202


In [56]:
testWithModel(DecisionTreeRegressor(criterion="absolute_error"),X_train, X_test, y_train, y_test)

MAE = 0.42613098831410445
MAPE = 0.13340288541534914
RMSE = 1.5133245313954877


In [57]:
testWithModel(ElasticNetCV(random_state=0), X_train, X_test, y_train, y_test)

MAE = 0.3117088740641135
MAPE = 0.1018202953851333
RMSE = 0.9970749589086461


In [58]:
testWithModel(SVR(C=300, epsilon=0.8),X_train, X_test, y_train, y_test)

MAE = 0.8800664313232881
MAPE = 0.2539063475139941
RMSE = 1.2163000915387363


Fill death col using random forest

In [59]:
null_death_rows['DEATH [d from CT]'] = clf_random_forest.predict(X2)
df = pd.concat([non_null_death_rows,null_death_rows])
y = df.iloc[:,11]
X = df.iloc[:, 0:11]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [60]:
print(X.shape)

(8878, 11)


In [61]:
X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size=0.2)

In [73]:
knn = KNeighborsRegressor(n_neighbors=9)
param_grid2 = dict(weights = ["uniform", "distance"])
knn_pred = testWithModel(GridSearchCV(knn, param_grid2),X_train, X_test, y_train, y_test)

MAE = 0.32587903017118763
MAPE = 0.07531791562156971
RMSE = 1.0034295397828479


In [74]:
svr_pred = testWithModel(SVR(C=300, epsilon=0.8),X_train, X_test, y_train, y_test)

MAE = 0.7166039116632771
MAPE = 0.16937317079943331
RMSE = 1.0875351905195174


In [75]:
lasso_pred = testWithModel(Lasso(alpha=0.1), X_train, X_test, y_train, y_test)

MAE = 0.28464332630668493
MAPE = 0.06456223731123001
RMSE = 0.9604603331617797


In [76]:
knn_pred = testWithModel(KNeighborsRegressor(),X_train, X_test, y_train, y_test)

MAE = 0.3460104406539516
MAPE = 0.07914148263712746
RMSE = 1.0575260535813897


In [77]:
elastic_net_pred = testWithModel(ElasticNetCV(random_state=0), X_train, X_test, y_train, y_test)

MAE = 0.2831924720771857
MAPE = 0.06418159019851027
RMSE = 0.9606106366747476


In [78]:
dtree_pred = testWithModel(DecisionTreeRegressor(criterion="absolute_error"),X_train, X_test, y_train, y_test)

MAE = 0.4162848843603914
MAPE = 0.09727644894780628
RMSE = 1.5069771406731205


Build Ensemble

In [79]:
array_preds = np.array([ dtree_pred, elastic_net_pred, knn_pred, lasso_pred,dtree_pred])
y_ens = np.mean((array_preds), axis=0 )

In [80]:
print(mean_absolute_error(y_ens, y_test))

0.3371806817215875


In [81]:
print(array_preds.shape)

(5, 1776)


In [82]:
import random
y_ens=[]

for i,y in enumerate(y_test):

  j = random.randint(0, 4)
  y_ens.append( array_preds[j][i])

In [83]:
len(y_ens)

1776

In [84]:
print(mean_absolute_error(y_ens, y_test))

0.35263111573870265
